In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import stats
import sklearn
from sklearn.preprocessing import KBinsDiscretizer

In [18]:
# Adding KBM value to df
# users = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/original_data/users_activated_v7.csv', low_memory = False)
df = pd.read_csv('C:/Users/sgulbin/Work/_Ad-hoc_analysis/usrs_w_no_deli_score.csv')
df['max_age'] = 50.0
df['max_exp'] = 20.0
df['age_limited'] = df[['max_age', 'age']].min(axis=1)
df['exp_limited'] = df[['max_exp', 'exp']].min(axis = 1)
# users = users[['user_id', 'sex', 'birthday', 'LicenseSetDate']]
# users.columns = ['user_id', 'sex', 'birthday', 'license_set_date']
# users = users[['user_id', 'user_ext','login', 'sex', 'birthday', 'license_set_date']]
# kbm = pd.read_csv('C:/Users/sgulbin/Work/Data/KBM_data/KBM_data_5_27_2020.csv')
# feb = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/FTR_score/users_feb/users_feb.csv', delimiter = ';')
# march = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/FTR_score/users_march/users_march.csv', delimiter = ';')
# df = pd.merge(pd.merge(march,users,left_on = 'user_ext', right_on = 'user_ext', how = 'left'), kbm, on = 'user_ext', how = 'left')
# df['user_id'] = df['user_id_y']
# df = pd.merge(users, kbm, on = 'user_id', how = 'left')
# df['age'] = np.clip((pd.to_datetime('today')-pd.to_datetime(df.birthday, errors = 'coerce')).astype('timedelta64[Y]'),19.0,80.0)
# df['exp'] = np.clip((pd.to_datetime('today')-pd.to_datetime(df.license_set_date, errors = 'coerce')).astype('timedelta64[Y]'),1.0,62.0)
# df['sex'] = df['sex'].fillna('Unknown')
df

,user_id,user_ext,login,activation_dtime,last_ride,status_ext,kbm,age,exp,sex,user_id.1,rents_w_ftr_score_only,rents_w_deli_score,max_ftr_score,min_ftr_score,need_ftr_calc,max_age,max_exp,age_limited,exp_limited
0,19500002,30,79153166530,2015-07-10 00:00:00,2019-12-23 10:32:05,2,0.65,34.0,4.0,M,NaN,NaN,NaN,NaN,NaN,ok,50.0,20.0,34.0,4.0
1,19500007,69,79166597585,2015-07-13 00:00:15,2019-07-13 18:44:53,2,0.70,52.0,23.0,F,NaN,NaN,NaN,NaN,NaN,ok,50.0,20.0,50.0,20.0
2,19500042,231,79055820281,2015-07-24 23:18:33,2017-07-06 19:22:05,2,0.70,52.0,7.0,M,NaN,NaN,NaN,NaN,NaN,ok,50.0,20.0,50.0,7.0
3,19500065,418,79250811456,2015-07-29 16:24:05,2020-02-05 17:45:24,2,0.50,41.0,18.0,M,NaN,NaN,NaN,NaN,NaN,ok,50.0,20.0,41.0,18.0
4,19500070,482,79060950706,2015-07-30 10:48:18,2019-06-14 19:27:05,2,0.90,30.0,12.0,M,NaN,NaN,NaN,NaN,NaN,ok,50.0,20.0,30.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804166,1582000035,32564321,79112329062,2020-09-25 11:09:13,2020-09-25 12:22:23,2,0.65,45.0,18.0,NaN,1.582000e+09,1.0,0.0,1.00,1.00,ok,50.0,20.0,45.0,18.0
804167,1582000077,32564684,79119406898,2020-09-25 11:48:02,2020-09-29 14:03:43,2,1.00,20.0,0.0,NaN,1.582000e+09,4.0,0.0,0.32,0.32,ok,50.0,20.0,20.0,0.0
804168,1582250060,32564451,79304091004,2020-09-25 11:33:02,NaN,2,0.50,44.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,ok,50.0,20.0,44.0,20.0
804169,1582250076,32564555,79257107739,2020-09-25 11:30:09,NaN,2,0.75,39.0,9.0,NaN,1.582250e+09,2.0,0.0,0.72,0.72,ok,50.0,20.0,39.0,9.0


In [19]:
# Generating age-exp coefficients
# K_age_exp_max = 1.2
# K_age_exp_min = 0.725
# K_intercept = -4.88430556234637
# K_f_age = 1.26397937674965
# K_f_exp = 0.0762464696907463
# f_age_exp_min = K_f_age*np.log(18)+K_f_exp*np.log(0.001)+K_intercept
# f_age_exp_max = K_f_age*np.log(50)+K_f_exp*np.log(20)+K_intercept
# K = K_age_exp_max-((K_f_age*np.log(18)+K_f_exp*np.log(0.001)+K_intercept)-f_age_exp_min)*(K_age_exp_max-K_age_exp_min)/(f_age_exp_max-f_age_exp_min)
# ages = []
# exps = []
# koefs = []
# for age in range(18,101):
#     for exp in range(0,83):
#         K = np.clip(K_age_exp_max-((K_f_age*np.log(age)+K_f_exp*np.log(exp)+K_intercept)-f_age_exp_min)*(K_age_exp_max-K_age_exp_min)/(f_age_exp_max-f_age_exp_min),K_age_exp_min,K_age_exp_max)
#         ages.append(age)
#         exps.append(exp)
#         koefs.append(K)
# age_exp_df = pd.DataFrame(data = ({'age':ages, 'exp':exps, 'K_age_exp': koefs}))
# test = age_exp_df.loc[(age_exp_df.age == 30)&(age_exp_df.exp == 0)]
# test
age_exp_df = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/coefficient_ftr_age_experience.csv')
age_exp_df

# print(age_exp_df)

,age,exp,K_age_exp
0,18,0,1.20
1,19,0,1.20
2,20,0,1.19
3,21,0,1.17
4,22,0,1.16
...,...,...,...
478,46,20,0.76
479,47,20,0.75
480,48,20,0.74
481,49,20,0.73


In [20]:
# Adding coefficients to df
df_K_sex = pd.DataFrame(data = ({'sex': ['M', 'F', np.nan], 'K_sex': [1.0,1.10,1.0]}))
df_K_KBM = pd.DataFrame(data = ({'KBM': [0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0,1.4,1.55,2.3,2.45,np.nan],
                                 'K_KBM': [0.9,0.9,0.9,0.9,0.9,0.9,0.95,0.95,1.0,1.0,1.03,1.1,1.1,1.1,1.1,1.03]}))
df = pd.merge(df, df_K_sex, how = 'left', left_on = 'sex', right_on = 'sex')
df = pd.merge(df, df_K_KBM, how = 'left', left_on = 'kbm', right_on = 'KBM')
df = pd.merge(df, age_exp_df, how = 'left', left_on = ['age_limited','exp_limited'], right_on = ['age','exp'])
# df['K_age_exp'] = np.where(df['age'].isnull(), 1.0, np.where(df['exp'].isnull(),1.0,np.where((df['age']-df['exp'])<16,1.0,df['K_age_exp'])))
df['K_age_exp'] = df['K_age_exp'].fillna(1.16)
# df['K_KBM'] = 1.03 #df['K_KBM'].fillna(1.0)

df['K_KBM_model_score'] = np.clip(df.K_sex*df.K_KBM*df.K_age_exp,0.7,1.299999)
df

,user_id,user_ext,login,activation_dtime,last_ride,status_ext,kbm,age_x,exp_x,sex,...,max_exp,age_limited,exp_limited,K_sex,KBM,K_KBM,age_y,exp_y,K_age_exp,K_KBM_model_score
0,19500002,30,79153166530,2015-07-10 00:00:00,2019-12-23 10:32:05,2,0.65,34.0,4.0,M,...,20.0,34.0,4.0,1.0,0.65,0.90,34.0,4.0,0.92,0.8280
1,19500007,69,79166597585,2015-07-13 00:00:15,2019-07-13 18:44:53,2,0.70,52.0,23.0,F,...,20.0,50.0,20.0,1.1,0.70,0.90,50.0,20.0,0.73,0.7227
2,19500042,231,79055820281,2015-07-24 23:18:33,2017-07-06 19:22:05,2,0.70,52.0,7.0,M,...,20.0,50.0,7.0,1.0,0.70,0.90,50.0,7.0,0.75,0.7000
3,19500065,418,79250811456,2015-07-29 16:24:05,2020-02-05 17:45:24,2,0.50,41.0,18.0,M,...,20.0,41.0,18.0,1.0,0.50,0.90,41.0,18.0,0.81,0.7290
4,19500070,482,79060950706,2015-07-30 10:48:18,2019-06-14 19:27:05,2,0.90,30.0,12.0,M,...,20.0,30.0,12.0,1.0,0.90,1.00,30.0,12.0,0.95,0.9500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804166,1582000035,32564321,79112329062,2020-09-25 11:09:13,2020-09-25 12:22:23,2,0.65,45.0,18.0,NaN,...,20.0,45.0,18.0,1.0,0.65,0.90,45.0,18.0,0.77,0.7000
804167,1582000077,32564684,79119406898,2020-09-25 11:48:02,2020-09-29 14:03:43,2,1.00,20.0,0.0,NaN,...,20.0,20.0,0.0,1.0,1.00,1.03,20.0,0.0,1.19,1.2257
804168,1582250060,32564451,79304091004,2020-09-25 11:33:02,NaN,2,0.50,44.0,26.0,NaN,...,20.0,44.0,20.0,1.0,0.50,0.90,44.0,20.0,0.78,0.7020
804169,1582250076,32564555,79257107739,2020-09-25 11:30:09,NaN,2,0.75,39.0,9.0,NaN,...,20.0,39.0,9.0,1.0,0.75,0.90,39.0,9.0,0.85,0.7650


In [21]:
# Mapping coefficient with score
mapa = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/FTR_score_mapping_v5.1.csv')
mapa.columns = ['ftr_score', 'from', 'to']
mapa.index=pd.IntervalIndex.from_arrays(left=mapa['from'],right=mapa['to'],closed= 'left')
df = df.assign(ftr_score=mapa.loc[df.K_KBM_model_score].ftr_score.values)
# df = df[['phone','ftr_score']]
# df.columns = ['user_id', 'user_ext', 'ftr_score']
df.to_csv('C:/Users/sgulbin/Work/_Ad-hoc_analysis/usrs_w_no_deli_score_ftr_calc.csv', index = False)
df

,user_id,user_ext,login,activation_dtime,last_ride,status_ext,kbm,age_x,exp_x,sex,...,age_limited,exp_limited,K_sex,KBM,K_KBM,age_y,exp_y,K_age_exp,K_KBM_model_score,ftr_score
0,19500002,30,79153166530,2015-07-10 00:00:00,2019-12-23 10:32:05,2,0.65,34.0,4.0,M,...,34.0,4.0,1.0,0.65,0.90,34.0,4.0,0.92,0.8280,0.82
1,19500007,69,79166597585,2015-07-13 00:00:15,2019-07-13 18:44:53,2,0.70,52.0,23.0,F,...,50.0,20.0,1.1,0.70,0.90,50.0,20.0,0.73,0.7227,0.99
2,19500042,231,79055820281,2015-07-24 23:18:33,2017-07-06 19:22:05,2,0.70,52.0,7.0,M,...,50.0,7.0,1.0,0.70,0.90,50.0,7.0,0.75,0.7000,1.00
3,19500065,418,79250811456,2015-07-29 16:24:05,2020-02-05 17:45:24,2,0.50,41.0,18.0,M,...,41.0,18.0,1.0,0.50,0.90,41.0,18.0,0.81,0.7290,0.98
4,19500070,482,79060950706,2015-07-30 10:48:18,2019-06-14 19:27:05,2,0.90,30.0,12.0,M,...,30.0,12.0,1.0,0.90,1.00,30.0,12.0,0.95,0.9500,0.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804166,1582000035,32564321,79112329062,2020-09-25 11:09:13,2020-09-25 12:22:23,2,0.65,45.0,18.0,NaN,...,45.0,18.0,1.0,0.65,0.90,45.0,18.0,0.77,0.7000,1.00
804167,1582000077,32564684,79119406898,2020-09-25 11:48:02,2020-09-29 14:03:43,2,1.00,20.0,0.0,NaN,...,20.0,0.0,1.0,1.00,1.03,20.0,0.0,1.19,1.2257,0.03
804168,1582250060,32564451,79304091004,2020-09-25 11:33:02,NaN,2,0.50,44.0,26.0,NaN,...,44.0,20.0,1.0,0.50,0.90,44.0,20.0,0.78,0.7020,1.00
804169,1582250076,32564555,79257107739,2020-09-25 11:30:09,NaN,2,0.75,39.0,9.0,NaN,...,39.0,9.0,1.0,0.75,0.90,39.0,9.0,0.85,0.7650,0.94


In [61]:
# Adding KBM value to dfr
usrs = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/original_data/users_activated_v6.csv', low_memory = False)
usrs = users[['user_id', 'login', 'sex', 'birthday', 'license_set_date']]
dfr = pd.merge(usrs, kbm, on = 'user_id', how = 'left')
dfr['age'] = np.clip((pd.to_datetime('today')-pd.to_datetime(dfr.birthday, errors = 'coerce')).astype('timedelta64[Y]'),19.0,80.0)
dfr['exp'] = np.clip((pd.to_datetime('today')-pd.to_datetime(dfr.license_set_date, errors = 'coerce')).astype('timedelta64[Y]'),1.0,62.0)
dfr['sex'] = dfr['sex'].fillna('Unknown')

# Adding coefficients to dfr
dfr_K_sex = pd.DataFrame(data = ({'sex': ['М', 'Ж', 'Unknown'], 'K_sex': [1.0,1.10,1.0]}))
dfr_K_KBM = pd.DataFrame(data = ({'KBM': [0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0,1.4,1.55,2.3,2.45,np.nan],
                                 'K_KBM': [0.9,0.9,0.9,0.9,0.9,0.9,0.95,0.95,1.0,1.0,1.03,1.1,1.1,1.1,1.1,1.03]}))
dfr = pd.merge(dfr, dfr_K_sex, how = 'left', left_on = 'sex', right_on = 'sex')
dfr = pd.merge(dfr, dfr_K_KBM, how = 'left', left_on = 'KBM', right_on = 'KBM')
dfr = pd.merge(dfr, age_exp_df, how = 'left', left_on = ['age','exp'], right_on = ['age','exp'])
dfr['K_age_exp'] = np.where(dfr['age'].isnull(), 1.0, np.where(dfr['exp'].isnull(),1.0,np.where((dfr['age']-dfr['exp'])<16,1.0,dfr['K_age_exp'])))
dfr['K_age_exp'] = dfr['K_age_exp'].fillna(1.0)
dfr['K_KBM'] = dfr['K_KBM'].fillna(1.0)
dfr['K_KBM_model_score'] = np.clip(dfr.K_sex*dfr.K_KBM*dfr.K_age_exp,0.7,1.3)

# Creating KBM_model_coefficient to KBM_model score mapping. Percentiles
est = KBinsDiscretizer(n_bins=103, encode='ordinal', strategy='quantile')
est.fit(dfr[['K_KBM_model_score']])
Xt = est.transform(dfr[['K_KBM_model_score']])
dfr['percentile'] = Xt
dfr



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_discretization.py:197: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)


,user_id,login,sex,birthday,license_set_date,user_ext,KBM,age,exp,K_sex,K_KBM,K_age_exp,K_KBM_model_score,percentile
0,19500001,79039686761,М,1981-11-13,2006-01-01,29.0,1.00,38.0,14.0,1.0,1.03,0.847423,0.872845,27.0
1,19500014,79856864080,М,1989-01-26,NaN,94.0,0.80,31.0,NaN,1.0,0.95,1.000000,0.950000,47.0
2,19500017,79857600842,Ж,1984-05-18,2005-01-01,101.0,0.90,35.0,15.0,1.1,1.00,0.879720,0.967692,51.0
3,19500021,79853192128,М,1981-08-09,1998-01-01,128.0,0.85,38.0,22.0,1.0,0.95,0.836144,0.794337,11.0
4,19500023,79262821237,М,1991-02-05,NaN,133.0,0.50,29.0,NaN,1.0,0.90,1.000000,0.900000,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842617,1202000120,79850089239,Ж,1997-12-17,2016-01-01,NaN,NaN,22.0,4.0,1.1,1.03,1.104764,1.251698,98.0
1842618,1202000172,79616095924,М,1981-01-07,1997-01-01,NaN,NaN,39.0,23.0,1.0,1.03,0.824290,0.849019,22.0
1842619,1202000178,79219234478,М,1995-03-22,2017-01-01,NaN,NaN,25.0,3.0,1.0,1.03,1.059064,1.090836,80.0
1842620,1202000182,79851190609,М,2001-04-04,2019-01-01,NaN,NaN,19.0,1.0,1.0,1.03,1.200000,1.236000,97.0


In [12]:
# test = pd.merge(df,dfr, on = 'user_id', how = 'left')
# test = test[['User_ext', 'percentile']]
# test.columns = ['User_ext', 'ftr_score']
# test.ftr_score = test.ftr_score/100.0
# test.to_csv('C:/Users/sgulbin/Work/_Выгрузки_и_Расчеты/users_for_ftr_score_output.csv')
np.log(0.001)

-6.907755278982137